In [1]:
!nvidia-smi

Thu Mar 14 06:18:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's de

In [3]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [4]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [5]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (151 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 121752 

In [6]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [8]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!rm -rf "db"

In [11]:
loader = PyPDFDirectoryLoader("/content/pdfs")
docs = loader.load()
len(docs)

452

In [12]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

1795

In [14]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 2min 46s, sys: 1.44 s, total: 2min 47s
Wall time: 3min 7s


In [15]:
model_name_or_path = "TheBloke/Llama-2-7B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

In [16]:
!nvidia-smi

Thu Mar 14 06:35:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              31W /  70W |   5375MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
DEFAULT_SYSTEM_PROMPT = """
Ask anything about LLAMA to the assistant""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [18]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

In [20]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [21]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end"
template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [22]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [24]:
result = qa_chain("how can we integrate computer vision with language models")

 Integrating computer vision with language models is an active area of research in natural language processing (NLP) and has numerous potential applications, including but not limited to:
1. Image captioning: Generating descriptive captions for images using language models that have been trained on large datasets of text and images.
2. Visual question answering: Answering questions about images by leveraging language models' ability to process and generate text.
3. Multimodal translation: Translating text into images or vice versa using language models that are capable of understanding both visual and linguistic representations.
4. Sentiment analysis: Analyzing the sentiment of images or videos using language models that have been trained on text data.
To achieve these integrations, researchers often use techniques such as image feature extraction, object detection, and attention mechanisms to focus on specific parts of the input data. Additionally, architectures like convolutional neu

In [25]:
len(result["source_documents"])

2

In [26]:
print(result["source_documents"][0].page_content)

[LC19] Guillaume Lample and Alexis Conneau. Cross-lingual language model pretraining. arXiv preprint
arXiv:1901.07291 , 2019.
70


In [27]:
result = qa_chain("What are the latest pre-trained LLM models")

 According to the text, the latest pre-trained LLM (Language Modeling) models are:
* BERT BASE with 110M parameters
* BERT LARGE with 340M parameters
* The largest model from a previous section with 64, 512, or 2 parameters
* Two additional MoE (Multi-Head Attention) models with high capacity (4 billion parameters) and higher computation budgets


In [28]:
print(result["source_documents"][0].page_content)

is (L=64, H=512, A=2) with 235M parameters
(Al-Rfou et al., 2018). By contrast, BERT BASE
contains 110M parameters and BERT LARGE con-
tains 340M parameters.
It has long been known that increasing the
model size will lead to continual improvements
on large-scale tasks such as machine translation
and language modeling, which is demonstrated
by the LM perplexity of held-out training data
shown in Table 6. However, we believe that
this is the ﬁrst work to demonstrate convinc-
ingly that scaling to extreme model sizes also
leads to large improvements on very small scale
tasks, provided that the model has been sufﬁ-
ciently pre-trained. Peters et al. (2018b) presented


In [29]:
result = qa_chain("Difference between llama and llama 2?")

 Sure! Here's an explanation of the difference between Llama and Llama 2 based on the provided context:
Llama and Llama 2 are two models developed by the same authors, but with slightly different architectures and training procedures. The main differences between them are as follows:
* Model size: Llama has a larger number of parameters (110M) compared to Llama 2 (44M). This is because Llama uses a larger BERT model architecture and more advanced training techniques, such as weight pruning and matrix factorization.
* Training procedure: Llama uses a combination of weight pruning and matrix factorization for optimization, while Llama 2 only uses weight pruning. Weight pruning involves removing redundant or unnecessary weights from the model, which can help reduce its size without significantly affecting its performance. Matrix factorization, on the other hand, involves representing the model's weights as a product of simpler matrices, which can also help reduce the model's size.
* Hyper

In [30]:
print(result["source_documents"][1].page_content)

LadaBERT-2×5.0×2.05 ×2.0×3.41
LadaBERT-3×7.5×5.0 ×2.0×4.33
LadaBERT-4×10.0×5.0 ×2.5×5.45
4.3 Performance Comparison
Table 3: Performance comparison on various model sizes
Algorithm MNLI-m MNLI-mm SST-2 QQP QNLI #Params Ratio
BERT-Base 84.6 83.4 93.5 71.2/- 90.5 110M×1.0
LadaBERT-1 83.5 82.5 92.8 70.7/88.9 89.6 44M×2.5
BERT-FT 74.8 74.3 86.4 65.8/86.9 84.3 44M×2.5
BERT-KD 75.4 74.8 86.9 67.3/87.6 84.0 44M×2.5
BERT-PKD 76.7 76.3 87.5 68.1/87.8 84.7 44M×2.5
Weight pruning 82.8 81.6 92.3 70.1/88.5 88.9 44M×2.5
matrix factorization 77.7 77.4 87.6 65.7/87.2 84.3 44M×2.5
Hybrid pruning 81.2 80.0 90.0 68.0/87.5 83.3 44M×2.5
LadaBERT-2 83.1 82.2 91.8 69.9/87.9 88.2 22M×5.0
Weight pruning 75.9 75.6 84.8 60.3/83.5 81.7 22M×5.0
matrix factorization 71.8 71.8 82.8 60.3/83.5 75.4 22M×5.0
Hybrid pruning 76.1 75.3 85.4 64.9/85.8 80.6 22M×5.0
LadaBERT-3 82.1 81.8 89.9 69.4/87.8 84.5 15M×7.5
TinyBERT 80.9 79.5 89.5 65.4/87.5 77.9 15M×7.5
BERT-Small 75.4 74.9 87.6 66.5/- 84.8 15M×7.5
